# Stable Diffusion notebook by [@alexpopo](https://github.com/alexpopo/alex_stable_diffusion_webui)

This let's you generate images with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/runwayml/stable-diffusion-v1-5).

Other Stable Diffusion tools:
* [Deforum Stable Diffusion](https://colab.research.google.com/github/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb) by [deforum_art](https://twitter.com/deforum_art) team
* [HuggingFace space for Stable Diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
* [Stable Diffusion Interpolation](https://colab.research.google.com/drive/1EHZtFjQoRr-bns1It5mTcOVyZzZD9bBc?usp=sharing) by [@ygantigravity](https://twitter.com/ygantigravity)
* [Stable Diffusion with diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57)
* [Prompt-space Interpolation](https://colab.research.google.com/drive/1gj8_EPyntZuaiQuzYOzog5YKA74esoB3#scrollTo=3g5rSm43STsW) by [@DigThatData](https://twitter.com/DigThatData)
* [Stable Diffusion Lite](https://colab.research.google.com/drive/1cl3d84B5AXepycKAfCba77faUFBRGY2O?usp=sharing#scrollTo=Rn06QzsdsICR) by [@future__art](https://twitter.com/future__art)
* [Stable Diffusion web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
* More to come!!

If you're looking for more Ai art tools check here [Ai generative art tools list](https://pharmapsychotic.com/tools.html).

if you encounter any issues, feel free to discuss them.[Discord Support](https://discord.com/channels/1002292111942635562)

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ad09a52b-a181-e21e-7cd5-fada40a99e84)


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Check Environment
#@markdown ##### 谷歌云端硬盘：仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘
#@markdown ##### 简单便捷方式：每次重新连接CoLab需要重新下载所有模型文件，不需要云硬盘

import os
import time

if os.path.exists("/content/gdrive/MyDrive/"):
  print('Gdrive connected, using Gdrive storage ...')
  print("仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘")
else:
  print('[1;31mGdrive not connected, using colab storage ...')
  print("每次重新连接CoLab需要重新下载所有模型文件")

time.sleep(1)

outputs_path = "/content/SDWebUI"
%mkdir -p $outputs_path
print()
%cd $outputs_path

Gdrive connected, using Gdrive storage ...
仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘

/content/SDWebUI


In [ ]:
#@title First Base Requirements

# 在尝试安装之前，请确保满足所有必需的依赖项
print("Check Common Software Versions ...")
!git --version
print()
print("git-v2.12及之后版本集成了git-lfs")
print("-"*80)
!apt --version
print("-"*80)
!python3 --version
print("-"*80)
!wget --version | grep "^GNU Wget"
print("-"*80)

print('Installing base requirements...')

# Debian-based:
!sudo apt install python3-venv

In [ ]:
#@title Next Base Requirements

import os

# Clone WebUI Repo Code And Download SD Model
# Clone WebUI Repo Code
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $outputs_path/stable-diffusion-webui


# Download Stable Diffusion Model Checkpoint Func
def download_sd_model(model_version="Default"):
  """
    parameter：
    model_version = [ "Default", "1.5-pruned", "V2.1-768px", "Anything-v4.5"]

    return:
     model_param
  """
  model_param = ""
  model_save_path = ""

  if os.path.exists("/content/gdrive/MyDrive/"):
    model_save_path = "/content/gdrive/MyDrive/AI/models/Stable-diffusion"
    %mkdir -p $model_save_path
    %cd $model_save_path
  else:
    model_save_path = "/content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion"
    %mkdir -p $model_save_path
    %cd $model_save_path

  if os.path.exists("/content/gdrive/MyDrive/"):
    if not os.path.exists("sd-v1-4.ckpt") and model_version=="Default":
      !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/sd-v1-4.ckpt
    elif not os.path.exists("v1-5-pruned-emaonly.ckpt") and model_version=="1.5-pruned":
      !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
    elif not os.path.exists("v2-1_768-ema-pruned.ckpt") and model_version=="V2.1-768px":
      !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt
    elif not os.path.exists("anything-v4.5-pruned.ckpt") and model_version=="Anything-v4.5":
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/anything-v4.5-pruned.ckpt
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/anything-v4.5-pruned.vae.pt
  else:
    if not os.path.exists("sd-v1-4.ckpt") and model_version=="Default":
      !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt
    elif not os.path.exists("v1-5-pruned-emaonly.ckpt") and model_version=="1.5-pruned":
      !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
    elif not os.path.exists("v2-1_768-ema-pruned.ckpt") and model_version=="V2.1-768px":
      !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt
    elif not os.path.exists("anything-v4.5-pruned.ckpt") and model_version=="Anything-v4.5":
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned.ckpt
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned.vae.pt

  if model_version=="Default":
    model_param = model_save_path + '/' + "sd-v1-4.ckpt"
  elif model_version=="1.5-pruned":
    model_param = model_save_path + '/' + "v1-5-pruned-emaonly.ckpt"
  elif model_version=="V2.1-768px":
    model_param = model_save_path + '/' + "v2-1_768-ema-pruned.ckpt"
  elif model_version=="Anything-v4.5":
    model_param = model_save_path + '/' + "anything-v4.5-pruned.ckpt"
  else:
    model_param = model_save_path + '/' + "model.ckpt"

  if model_version=="V2.1-768px":
    %cd $model_save_path
    !git clone https://github.com/Stability-AI/stablediffusion.git $outputs_path/stablediffusion
    !mv -f /content/SDWebUI/stablediffusion/configs/stable-diffusion/v2-inference-v.yaml v2-1_768-ema-pruned.yaml
    !rm -r /content/SDWebUI/stablediffusion

  return model_param


"""
Git LFS Details
SHA256: fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556
Pointer size: 135 Bytes
Size of remote file: 4.27 GB
"""
# 下载模型文件"sd-v1-4.ckpt"并进行hash校验，检查是否下载完整
if os.path.exists("/content/gdrive/MyDrive/"):
  %mkdir -p /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %cd /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %rm -r /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion
  !ln -s /content/gdrive/MyDrive/AI/models/Stable-diffusion /content/SDWebUI/stable-diffusion-webui/models
  if not os.path.exists("sd-v1-4.ckpt"):
    download_sd_model()
    !sha256sum sd-v1-4.ckpt
else:
  download_sd_model()
  !sha256sum $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt

Cloning into '/content/SDWebUI/stable-diffusion-webui'...
remote: Enumerating objects: 16079, done.
remote: Total 16079 (delta 0), reused 0 (delta 0), pack-reused 16079
Receiving objects: 100% (16079/16079), 27.23 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (11292/11292), done.
/content/gdrive/MyDrive/AI/models/Stable-diffusion


In [ ]:
#@title Automatic Installation (Take a ten minute break with a cup of tea)

%cd $outputs_path/stable-diffusion-webui

!sed -i 's@#install_dir="/home/$(whoami)"@install_dir="/content/SDWebUI"@' $outputs_path/stable-diffusion-webui/webui-user.sh
!sed -i "s@can_run_as_root=0@can_run_as_root=1@" $outputs_path/stable-diffusion-webui/webui.sh

# [Bug]: fastapi 0.91 app.add_middleware #7714
# [Fix]: Roll back the previous version #7717
!sed -i "/httpcore<=0.15/afastapi==0.90.1" $outputs_path/stable-diffusion-webui/requirements_versions.txt

!bash webui.sh

/content/SDWebUI/stable-diffusion-webui

################################################################
Install script for stable-diffusion + Web UI
Tested on Debian 11 (Bullseye)
################################################################

################################################################
Running on root user
################################################################

################################################################
Repo already cloned, using it as install directory
################################################################

################################################################
Create and activate python venv
################################################################

################################################################
Launching launch.py...
################################################################
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: ea9bd9fc7409109adcd61b897abc2

In [ ]:
#@title WebUI自动启动成功会有下面提示，停止上面的自动运行，安装'xformers'模块
# Running on local URL:  http://127.0.0.1:7860
# To create a public link, set `share=True` in `launch()`.

# Launching Web UI with arguments: No module 'xformers' Issues.
# fix bug: install xformers-0.0.16-cp38-cp38-manylinux2014_x86_64.whl
%cd /content/SDWebUI/stable-diffusion-webui

!source venv/bin/activate; python3 -m pip install xformers==0.0.16; python3 -m pip list | grep -e "xformers"

/content/SDWebUI/stable-diffusion-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50.9 MB 59 kB/s 
xformers                0.0.16         


### Prompt Demo

* green sapling rowing out of ground, mud, dirt, grass, high quality, photorealistic, sharp focus, depth of field

* Steps: 20, Sampler: Euler a, CFG scale: 12, Seed: 1441787169, Size: 512x512, Model hash: fe4efff1e1, Model: sd-v1-4

In [ ]:
#@title Restart Stable-Diffusion
import os

#@markdown Model Download/Load Checkpoint
Set_Model_Version = "V2.1-768px" #@param [ "Default", "1.5-pruned", "V2.1-768px", "Anything-v4.5"]
model_param = download_sd_model(Set_Model_Version)

#@markdown Add credentials to your Gradio interface (optional).
User = "alexpopo" #@param {type:"string"}
Password= "123456" #@param {type:"string"}

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

%cd $outputs_path/stable-diffusion-webui

try:
  if os.path.isfile(model_param):
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="dark" --ckpt "$model_param"
  else:
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="light" --ckpt-dir "$model_param"
except:
  !source venv/bin/activate; python3 webui.py --share --xformers $auth

/content/gdrive/MyDrive/AI/models/Stable-diffusion
/content/gdrive/MyDrive/AI/models/Stable-diffusion
Cloning into '/content/SDWebUI/stablediffusion'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 301 (delta 122), reused 242 (delta 108), pack-reused 0
Receiving objects: 100% (301/301), 68.92 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/SDWebUI/stable-diffusion-webui
Calculating sha256 for /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt: ad2a33c361c1f593c4a1fb32ea81afce2b5bb7d1983c6b94793a26a3b54b08a0
Loading weights [ad2a33c361] from /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt
Creating model from config: /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml
LatentDiffusion: Running in v-prediction mode
DiffusionWrapper has 865.91 M